## 이미지 NIFTI <===> NUMPY 변환

In [1]:
# label 확인용
import shutil
shutil.make_archive('nifti_label', 'zip', './seg_nifti_300/')

'/home/jhj/Desktop/HJ_RESEARCH/snu_brain_disease/nifti_label.zip'

---

In [1]:
# NIFTI ---> NUMPY
import pandas as pd
import nibabel as nib
import numpy as np
import os
from tqdm import tqdm

img_dir = './input_nifti_300/'
save_path_np = './input_numpy_300/'

df = pd.read_csv('labels/data_975.csv', index_col=0)
filenames = df.index

if not os.path.isdir(save_path_np):
    os.mkdir(save_path_np)

for filename in tqdm(filenames):
    # load nifti image
    img = nib.load(img_dir + filename + '.nii.gz')
    img = img.get_fdata()
    # save as numpy
    np.save(save_path_np + filename, img)

100%|█████████████████████████████████████████| 975/975 [19:34<00:00,  1.20s/it]


---

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
"""
CROP FIRST THEN RESIZE
[ dim x pixdim = mm ]
"""
from scipy.ndimage import zoom

img_dir = './input_nifti/'
img_dir2 = './test_nifti/'
label_dir = 'labels/data_975.csv'
save_path_img = './input_nifti_300/'

df = pd.read_csv(label_dir, index_col=0)

img_path_list = df.index

if not os.path.isdir(save_path_img):
    os.mkdir(save_path_img)

for index in tqdm(img_path_list):
    path = img_dir + index + '.nii.gz'
    
    if os.path.isfile(save_path_img + index + '.nii.gz'):
        continue
        
    if not os.path.isfile(path):
        path = img_dir2 + index + '.nii.gz'

    img = nib.load(path)
    dim = img.header['dim']
    pixdim = img.header['pixdim']
    img = img.get_fdata()
        
    scale_x = 300 / dim[1]
    scale_y = 300 / dim[2]
    scale_z = 300 / dim[3]

    img = zoom(img, (scale_x, scale_y, scale_z))
            
#    # check
#     print(img.shape)
    
#     plt.subplot(1, 3, 1)
#     plt.title('coronal')
#     plt.imshow(np.max(img, axis=1), cmap='gray')
    
#     plt.subplot(1, 3, 2)
#     plt.title('sagittal')
#     plt.imshow(img[img.shape[0]//2, :, :], cmap='gray')
    
#     plt.subplot(1, 3, 3)
#     plt.title('axial')
#     plt.imshow(img[:, :, img.shape[2]//2], cmap='gray')
    
#     plt.show()
#     plt.close()
#     break
    
    # save as nib
    nifti_img = nib.Nifti1Image(img, affine=np.eye(4))
    nib.save(nifti_img, save_path_img + index + '.nii.gz')

100%|█████████████████████████████████████████| 975/975 [41:30<00:00,  2.55s/it]


---

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
"""
CROP FIRST THEN RESIZE
[ dim x pixdim = mm ]
"""
from scipy.ndimage import zoom

img_dir = './input_nifti/'
seg_dir = './input_nifti_seg/'
save_path_img = './input_nifti_300/'
save_path_seg = './seg_nifti_300/'

img_path_list = os.listdir(seg_dir)

if not os.path.isdir(save_path_img):
    os.mkdir(save_path_img)
if not os.path.isdir(save_path_seg):
    os.mkdir(save_path_seg)

for img_path in tqdm(img_path_list):
    index = img_path.split('.')[0]
    path = img_dir + img_path
    seg_path = seg_dir + img_path
    
    if os.path.isfile(save_path_seg + index + '.nii.gz'):
        continue

    img = nib.load(path)
    dim = img.header['dim']
    pixdim = img.header['pixdim']
    img = img.get_fdata()
    
    seg = nib.load(seg_path)
    seg = seg.get_fdata()
        
    scale_x = 300 / dim[1]
    scale_y = 300 / dim[2]
    scale_z = 300 / dim[3]

    img = zoom(img, (scale_x, scale_y, scale_z))
    seg = zoom(seg, (scale_x, scale_y, scale_z))
            
#    # check
#     print(img.shape)
    
#     plt.subplot(1, 3, 1)
#     plt.title('coronal')
#     plt.imshow(np.max(img, axis=1), cmap='gray')
    
#     plt.subplot(1, 3, 2)
#     plt.title('sagittal')
#     plt.imshow(img[img.shape[0]//2, :, :], cmap='gray')
    
#     plt.subplot(1, 3, 3)
#     plt.title('axial')
#     plt.imshow(img[:, :, img.shape[2]//2], cmap='gray')
    
#     plt.show()
#     plt.close()
#     break
    
    # save as nib
    nifti_img = nib.Nifti1Image(img, affine=np.eye(4))
    nib.save(nifti_img, save_path_img + index + '.nii.gz')
    nifti_seg = nib.Nifti1Image(seg, affine=np.eye(4))
    nib.save(nifti_seg, save_path_seg + index + '.nii.gz')

100%|█████████████████████████████████████████| 130/130 [07:00<00:00,  3.24s/it]


---

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
"""
CROP FIRST THEN RESIZE
[ dim x pixdim = mm ]
"""
from scipy.ndimage import zoom

df1 = pd.read_csv('./labels/bifurcation_zyx.csv', index_col=0)
df2 = pd.read_csv('./labels/bifurcation_zyx_new.csv', index_col=0)
df3 = pd.read_csv('./labels/test_bifurcation_zyx.csv', index_col=0)
index1 = df1.index
index2 = df2.index
index3 = df3.index

img_dir = './input_nifti_seg/'
save_path = './seg_nifti_crop/'
save_path_np = './seg_np/'

img_path_list = os.listdir(img_dir)

if not os.path.isdir(save_path):
    os.mkdir(save_path)
if not os.path.isdir(save_path_np):
    os.mkdir(save_path_np)

for img_path in tqdm(img_path_list):
    index = img_path.split('.')[0]
    path = img_dir + img_path
    
    if os.path.isfile(save_path + index + '.npy'):
        continue
        
    if index in index1:
        df = df1
    if index in index2:
        df = df2
    elif index in index3:
        df = df3
        
    x = (df.loc[index]['left_x'] + df.loc[index]['right_x']) // 2
    y = (df.loc[index]['left_y'] + df.loc[index]['right_y']) // 2
    z = df.loc[index]['mid_z']

    img = nib.load(path)
    pixdim = img.header['pixdim']
    img = img.get_fdata()
        
    # RL: 100, SI: 100, AP 50 [mm]
    crop_x = int(100 / pixdim[1]) // 2
    crop_y = int(50 / pixdim[2]) // 2
    crop_z = int(100 / pixdim[3]) // 2
    img_crop = img[x-crop_x:x+crop_x, y-crop_y:y+crop_y, z-crop_z:z+crop_z]
    img = img_crop
    
    if img.shape[2] != 98:
        print('resampling...')
        scale = 98 / img.shape[2]
        img = zoom(img, (1, 1, scale))
            
#    # check
#     print(img.shape)
    
#     plt.subplot(1, 3, 1)
#     plt.title('coronal')
#     plt.imshow(np.max(img, axis=1), cmap='gray')
    
#     plt.subplot(1, 3, 2)
#     plt.title('sagittal')
#     plt.imshow(img[img.shape[0]//2, :, :], cmap='gray')
    
#     plt.subplot(1, 3, 3)
#     plt.title('axial')
#     plt.imshow(img[:, :, img.shape[2]//2], cmap='gray')
    
#     plt.show()
#     plt.close()
#     break

    # save as numpy
    np.save(save_path_np + index, img)
    
    # save as nib
    nifti_img = nib.Nifti1Image(img, affine=np.eye(4))
    nib.save(nifti_img, save_path + index + '.nii.gz')

  3%|█                                          | 3/115 [00:01<00:53,  2.09it/s]

resampling...


  7%|██▉                                        | 8/115 [00:04<00:56,  1.88it/s]

resampling...


 31%|█████████████▏                            | 36/115 [00:18<00:37,  2.09it/s]

...

 97%|███████████████████████████████████████▌ | 111/115 [01:00<00:02,  1.93it/s]

resampling...


 97%|███████████████████████████████████████▉ | 112/115 [01:01<00:01,  1.51it/s]

resampling...


100%|█████████████████████████████████████████| 115/115 [01:03<00:00,  1.82it/s]


---

In [2]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [8]:
"""
CROP FIRST THEN RESIZE
[ dim x pixdim = mm ]
"""
from scipy.ndimage import zoom

df1 = pd.read_csv('./labels/bifurcation_zyx.csv', index_col=0)
df2 = pd.read_csv('./labels/bifurcation_zyx_new.csv', index_col=0)
df3 = pd.read_csv('./labels/test_bifurcation_zyx.csv', index_col=0)
index1 = df1.index
index2 = df2.index
index3 = df3.index

img_dir = './input_nifti_seg/'  # label 경로
input_dir = './input_nifti/'  # input 경로
save_path = './input_nifti_crop/'
# save_path_np = './input_np/'

img_path_list = os.listdir(img_dir)

if not os.path.isdir(save_path):
    os.mkdir(save_path)
# if not os.path.isdir(save_path_np):
#     os.mkdir(save_path_np)

for img_path in tqdm(img_path_list):
    index = img_path.split('.')[0]
    path = input_dir + img_path
    
#     if os.path.isfile(save_path + index + '.nii.gz') and os.path.isfile(save_path_np + index + '.npy'):
#         continue
        
    if index in index1:
        df = df1
    if index in index2:
        df = df2
    elif index in index3:
        df = df3
        
    x = (df.loc[index]['left_x'] + df.loc[index]['right_x']) // 2
    y = (df.loc[index]['left_y'] + df.loc[index]['right_y']) // 2
    z = df.loc[index]['mid_z']

    img = nib.load(path)
    pixdim = img.header['pixdim']
    img = img.get_fdata()
        
    # RL: 100, SI: 100, AP 50 [mm]
    crop_x = int(100 / pixdim[1]) // 2
    crop_y = int(50 / pixdim[2]) // 2
    crop_z = int(100 / pixdim[3]) // 2
    img_crop = img[x-crop_x:x+crop_x, y-crop_y:y+crop_y, z-crop_z:z+crop_z]
    img = img_crop
    
    if img.shape[2] != 98:
        print('resampling...')
        scale = 98 / img.shape[2]
        img = zoom(img, (1, 1, scale))
            
#    # check
#     print(img.shape)
    
#     plt.subplot(1, 3, 1)
#     plt.title('coronal')
#     plt.imshow(np.max(img, axis=1), cmap='gray')
    
#     plt.subplot(1, 3, 2)
#     plt.title('sagittal')
#     plt.imshow(img[img.shape[0]//2, :, :], cmap='gray')
    
#     plt.subplot(1, 3, 3)
#     plt.title('axial')
#     plt.imshow(img[:, :, img.shape[2]//2], cmap='gray')
    
#     plt.show()
#     plt.close()
#     break

#     # save as numpy
#     np.save(save_path_np + index, img)
    
    # save as nib
    nifti_img = nib.Nifti1Image(img, affine=np.eye(4))
    nib.save(nifti_img, save_path + index + '.nii.gz')

  3%|█                                          | 3/115 [00:01<00:59,  1.88it/s]

resampling...


  7%|██▉                                        | 8/115 [00:04<01:01,  1.74it/s]

resampling...


 31%|█████████████▏                            | 36/115 [00:20<00:44,  1.78it/s]

...

 97%|███████████████████████████████████████▌ | 111/115 [01:07<00:02,  1.79it/s]

resampling...


 97%|███████████████████████████████████████▉ | 112/115 [01:08<00:02,  1.38it/s]

resampling...


100%|█████████████████████████████████████████| 115/115 [01:10<00:00,  1.63it/s]


---

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
from tqdm import tqdm

In [2]:
label_name = 'FL_WMH_VOL_icv'
# label_name = 'FL_WMH_VOL_mL'
# label_name = 'WMH_GRADE'
# label_name = 'Cortical Gray Matter Total Percent Of Icv'
# label_name = 'FL_PVWMH_VOL_icv'
# label_name = 'FL_DWMH_VOL_icv'
img_dir = 'img_npy/'
label_dir = 'labels/data_975.csv'

In [3]:
nifti_dir = 'cropped_nifti'

In [ ]:
df = pd.read_csv(label_dir, index_col=0)[label_name]
filenames = df.index

if not os.path.isdir(nifti_dir):
    os.mkdir(nifti_dir)

for i, index in tqdm(enumerate(filenames)):
    file_name = img_dir + index + '.npy'
    img = np.load(file_name)
    nifti_img = nib.Nifti1Image(img, affine=np.eye(4))
    nib.save(nifti_img, nifti_dir + '/' + index + '.nii')

381it [00:06, 61.30it/s]

---

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
from tqdm import tqdm
from scipy.ndimage import zoom

In [2]:
# Save Resized Numpy Arrays as NIFTI
root = 'input_nifti_resized/'
target = 'input_nifti_256/'

dir_names = os.listdir(root)
# print(len(dir_names))  # num of files in input_nifti/
# print(dir_names)

if not os.path.isdir(target):
    os.mkdir(target)

count = 0
for i, dirname in enumerate(tqdm(dir_names)):
    if "MR" not in dirname:
        print("filename error: " + dirname)
        continue
    file_name = root + dirname
    new_file_name = target + dirname
    if os.path.isfile(new_file_name): continue
    
    img = nib.load(file_name)
    img_data = img.get_fdata()
    img = np.transpose(img_data, (2, 1, 0))
    
    ni_img = nib.Nifti1Image(img, affine=np.eye(4))
    nib.save(ni_img, new_file_name)

 38%|███████████████▌                         | 306/805 [24:15<40:02,  4.81s/it]

filename error: .ipynb_checkpoints


100%|███████████████████████████████████████| 805/805 [1:04:08<00:00,  4.78s/it]


In [ ]:
# Save Resized Numpy Arrays as NIFTI
root = 'test_nifti/'
target = 'test_nifti_256/'

dir_names = os.listdir(root)
# print(len(dir_names))  # num of files in input_nifti/
# print(dir_names)

if not os.path.isdir(target):
    os.mkdir(target)

count = 0
for i, dirname in enumerate(tqdm(dir_names)):
    file_name = root + dirname + '/' + os.listdir(root + dirname)[0]
    new_file_name = target + dirname + '.nii.gz'
    if os.path.isfile(new_file_name): continue
    
    img = nib.load(file_name)
    img_data = img.get_fdata()
#     img = np.transpose(img_data, (2, 1, 0))
    
    # resample (resize)
    scale1 = 256 / img.shape[0]
    scale2 = 256 / img.shape[1]
    scale3 = 256 / img.shape[2]
    img = zoom(img, zoom=(scale1, scale2, scale3))
    
    ni_img = nib.Nifti1Image(img, affine=np.eye(4))
    nib.save(ni_img, new_file_name)